In [ ]:
!pip install gradio transformers torch gtts SpeechRecognition pydub

import gradio as gr
import speech_recognition as sr
from gtts import gTTS
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import os

# تحميل النموذج المجاني من Microsoft
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-small")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-small")

# تخزين المحادثة
chat_history_ids = None

# دالة الرد
def chat_with_bot(user_input):
    global chat_history_ids

    new_input_ids = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors='pt')

    if chat_history_ids is not None:
        bot_input_ids = torch.cat([chat_history_ids, new_input_ids], dim=-1)
    else:
        bot_input_ids = new_input_ids

    chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)
    reply = tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)
    return reply

# دالة تحويل الصوت إلى نص + الرد الصوتي
def process_interaction(audio_file=None, text_input=""):
    final_input = ""

    if audio_file:
        try:
            recognizer = sr.Recognizer()
            with sr.AudioFile(audio_file) as source:
                audio = recognizer.record(source)
            final_input = recognizer.recognize_google(audio, language='ar-TN')
        except sr.UnknownValueError:
            return "🛑 لم أفهم صوتك. جرب تكتب تحت.", None
        except sr.RequestError as e:
            return f"⚠️ خطأ في الاتصال: {e}", None

    if text_input.strip() != "":
        final_input = text_input.strip()

    if final_input == "":
        return "🟡 لم يتم إدخال صوت أو نص!", None

    # الرد من الشاتبوت
    reply = chat_with_bot(final_input)

    # تحويل الرد إلى صوت
    tts = gTTS(text=reply, lang='ar')
    tts.save("reply.mp3")

    return reply, "reply.mp3"

# 🎛️ الواجهة الاحترافية
interface = gr.Interface(
    fn=process_interaction,
    inputs=[
        gr.Audio(source="microphone", type="filepath", label="🎙️ تكلم إذا تحب"),
        gr.Textbox(lines=2, placeholder="أو اكتب هنا...", label="⌨️ الكتابة")
    ],
    outputs=[
        gr.Textbox(label="📄 الرد الكتابي"),
        gr.Audio(label="🔊 الرد الصوتي")
    ],
    title="🤖 شاتبوت بالصوت والكتابة - مجاني 100%",
    description="احكي أو اكتب بالعربية / التونسية / الفرنسية، والشاتبوت يرد عليك صوتًا وكتابة. بدون API وبدون فلوس 💸."
)

interface.launch(share=True)


TypeError: Audio.__init__() got an unexpected keyword argument 'source'

In [ ]:
!pip install gradio transformers torch gtts SpeechRecognition pydub

import gradio as gr
import speech_recognition as sr
from gtts import gTTS
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import os
from pydub import AudioSegment

# تحميل نموذج DialoGPT الصغير
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-small")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-small")

# حفظ المحادثة
chat_history_ids = None

# دالة المحادثة
def chat_with_bot(user_input):
    global chat_history_ids
    new_input_ids = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors='pt')

    if chat_history_ids is not None:
        bot_input_ids = torch.cat([chat_history_ids, new_input_ids], dim=-1)
    else:
        bot_input_ids = new_input_ids

    chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)

    reply = tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)
    return reply

# دالة تحويل الصوت إلى نص + الرد الصوتي
def process_interaction(audio_file=None, text_input=""):
    final_input = ""

    if audio_file:
        try:
            recognizer = sr.Recognizer()
            with sr.AudioFile(audio_file) as source:
                audio = recognizer.record(source)
            final_input = recognizer.recognize_google(audio, language='ar-TN')
        except sr.UnknownValueError:
            return "🛑 لم أفهم صوتك. جرب تكتب تحت.", None
        except sr.RequestError as e:
            return f"⚠️ خطأ في الاتصال: {e}", None

    if text_input.strip() != "":
        final_input = text_input.strip()

    if final_input == "":
        return "🟡 لم يتم إدخال صوت أو نص!", None

    # الرد من الشاتبوت
    reply = chat_with_bot(final_input)

    # تحويل الرد إلى صوت
    tts = gTTS(text=reply, lang='ar')
    tts.save("reply.mp3")

    return reply, "reply.mp3"

# واجهة Gradio
gr.Interface(
    fn=process_interaction,
    inputs=[
        gr.Audio(type="filepath", label="🎙️ تكلم إذا تحب"),
        gr.Textbox(lines=2, placeholder="أو اكتب هنا...", label="⌨️ الكتابة")
    ],
    outputs=[
        gr.Textbox(label="📄 الرد الكتابي"),
        gr.Audio(label="🔊 الرد الصوتي")
    ],
    title="ChatBot ",
    description="AI."
).launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://381fd23764e808be07.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
